# PhysBound Demo: Physics Validation in Action

This notebook demonstrates PhysBound's physics engines directly. In production, these are exposed as MCP tools — here we call them as Python functions to show what's happening under the hood.

In [ ]:
from physbound.engines.shannon import channel_capacity_bps, snr_db_to_linear, validate_throughput_claim
from physbound.engines.link_budget import compute_link_budget, free_space_path_loss_db, max_aperture_gain_dbi
from physbound.engines.noise import thermal_noise_power_dbm, friis_noise_cascade, receiver_sensitivity_dbm
from physbound.errors import PhysicalViolationError

## 1. Shannon-Hartley: Can 20 MHz Wi-Fi do 500 Mbps?

LLMs routinely claim impossible throughputs. Let's check a 20 MHz 802.11n channel at 15 dB SNR.

In [ ]:
bandwidth = 20e6   # 20 MHz
snr = snr_db_to_linear(15.0)  # 15 dB

capacity = channel_capacity_bps(bandwidth, snr)
print(f"Shannon capacity: {capacity / 1e6:.1f} Mbps")
print(f"LLM claimed:      500.0 Mbps")
print(f"Excess:           {(500e6 / capacity - 1) * 100:.1f}%")

In [ ]:
# PhysBound catches the impossible claim
try:
    validate_throughput_claim(bandwidth, snr, claimed_throughput_bps=500e6)
except PhysicalViolationError as e:
    print(f"CAUGHT: {e.message}")
    print(f"Law:    {e.law_violated}")
    print(f"LaTeX:  {e.latex_explanation}")

## 2. Antenna Aperture: Can a 30 cm dish do 45 dBi at 1 GHz?

Antenna gain is bounded by the aperture limit: $G_{\max} = \eta \left(\frac{\pi D}{\lambda}\right)^2$

In [ ]:
g_max = max_aperture_gain_dbi(diameter_m=0.3, frequency_hz=1e9)
print(f"Max gain for 0.3 m dish at 1 GHz: {g_max:.1f} dBi")
print(f"LLM claimed:                       45.0 dBi")
print(f"Off by:                            {45.0 - g_max:.1f} dB (a factor of {10**((45.0 - g_max)/10):.0f}x)")

In [ ]:
# PhysBound rejects the impossible gain in a link budget
try:
    compute_link_budget(
        tx_power_dbm=20, tx_antenna_gain_dbi=45, rx_antenna_gain_dbi=0,
        frequency_hz=1e9, distance_m=1000, tx_antenna_diameter_m=0.3,
    )
except PhysicalViolationError as e:
    print(f"CAUGHT: {e.message}")
    print(f"Law:    {e.law_violated}")

## 3. Thermal Noise Floor

The noise floor at 290K is fixed by the Boltzmann constant: $N = k_B T B = -174 \text{ dBm/Hz}$

In [ ]:
# 1 Hz bandwidth at 290K — the fundamental reference
n_1hz = thermal_noise_power_dbm(1.0, 290.0)
print(f"Noise floor at 290K, 1 Hz:   {n_1hz:.2f} dBm/Hz")
print(f"LLM hallucination:           -180.00 dBm/Hz")
print(f"Error:                       {abs(n_1hz - (-180)):.2f} dB\n")

# Scale to practical bandwidths
for bw_label, bw in [("1 kHz", 1e3), ("1 MHz", 1e6), ("20 MHz", 20e6)]:
    n = thermal_noise_power_dbm(bw, 290.0)
    print(f"Noise floor at 290K, {bw_label:>6s}: {n:.2f} dBm")

## 4. Receiver Cascade: Why LNA Order Matters

The Friis noise cascade formula shows that the first stage dominates system noise figure.

In [ ]:
# Good order: LNA first (low NF, high gain), then mixer
stages_good = [(20.0, 1.5), (10.0, 8.0)]  # (gain_db, nf_db)
nf_good = friis_noise_cascade(stages_good)

# Bad order: mixer first, then LNA
stages_bad = [(10.0, 8.0), (20.0, 1.5)]
nf_bad = friis_noise_cascade(stages_bad)

print(f"LNA first → mixer: NF = {nf_good:.2f} dB")
print(f"Mixer first → LNA: NF = {nf_bad:.2f} dB")
print(f"Penalty for wrong order: {nf_bad - nf_good:.2f} dB")

In [ ]:
# Impact on receiver sensitivity (10 MHz BW, 10 dB required SNR)
sens_good = receiver_sensitivity_dbm(10e6, nf_good, 10.0)
sens_bad = receiver_sensitivity_dbm(10e6, nf_bad, 10.0)

print(f"Sensitivity (LNA first):  {sens_good:.2f} dBm")
print(f"Sensitivity (mixer first): {sens_bad:.2f} dBm")
print(f"Lost sensitivity:          {sens_bad - sens_good:.2f} dB")

## 5. Link Budget: Wi-Fi to GEO Satellite

Free-space path loss makes long-distance links much harder than LLMs suggest.

In [ ]:
# Wi-Fi at 10 km — LLM claims -40 dBm received
wifi = compute_link_budget(
    tx_power_dbm=20, tx_antenna_gain_dbi=3, rx_antenna_gain_dbi=3,
    frequency_hz=2.4e9, distance_m=10_000,
)
print(f"Wi-Fi at 10 km:")
print(f"  FSPL:     {wifi['fspl_db']:.1f} dB")
print(f"  RX power: {wifi['received_power_dbm']:.1f} dBm (LLM claimed -40 dBm)")
print()

# 1W to GEO — LLM claims -80 dBm received
geo = compute_link_budget(
    tx_power_dbm=30, tx_antenna_gain_dbi=0, rx_antenna_gain_dbi=0,
    frequency_hz=12e9, distance_m=35_786_000,
)
print(f"1W to GEO (12 GHz, 0 dBi antennas):")
print(f"  FSPL:     {geo['fspl_db']:.1f} dB")
print(f"  RX power: {geo['received_power_dbm']:.1f} dBm (LLM claimed -80 dBm)")